# Advanced Topics - MultiIndexing

Up until now, the data we've worked with has been 2D. Just a table, that you could write down on a piece of paper. But there's no need to limit ourselves to that!

For this example, let's use a big dataset - 2015 flight delays from here:
https://www.kaggle.com/usdot/flight-delays

Because the file is so large, I've also gzipped it and only taken the first month. Note that with pandas you dont need to unzip the file to read it!

In [2]:
import pandas as pd

dfo = pd.read_csv("flights2.csv.gz", low_memory=False)
print(dfo.columns)
dfo.head()

Index(['YEAR', 'MONTH', 'DAY', 'DAY_OF_WEEK', 'AIRLINE', 'FLIGHT_NUMBER',
       'TAIL_NUMBER', 'ORIGIN_AIRPORT', 'DESTINATION_AIRPORT',
       'SCHEDULED_DEPARTURE', 'DEPARTURE_TIME', 'DEPARTURE_DELAY', 'TAXI_OUT',
       'WHEELS_OFF', 'SCHEDULED_TIME', 'ELAPSED_TIME', 'AIR_TIME', 'DISTANCE',
       'WHEELS_ON', 'TAXI_IN', 'SCHEDULED_ARRIVAL', 'ARRIVAL_TIME',
       'ARRIVAL_DELAY', 'DIVERTED', 'CANCELLED', 'CANCELLATION_REASON',
       'AIR_SYSTEM_DELAY', 'SECURITY_DELAY', 'AIRLINE_DELAY',
       'LATE_AIRCRAFT_DELAY', 'WEATHER_DELAY'],
      dtype='object')


YEAR  MONTH  DAY  DAY_OF_WEEK AIRLINE  FLIGHT_NUMBER TAIL_NUMBER  \
0  2015      1    1            4      AS             98      N407AS   
1  2015      1    1            4      AA           2336      N3KUAA   
2  2015      1    1            4      US            840      N171US   
3  2015      1    1            4      AA            258      N3HYAA   
4  2015      1    1            4      AS            135      N527AS   

  ORIGIN_AIRPORT DESTINATION_AIRPORT  SCHEDULED_DEPARTURE  ...  ARRIVAL_TIME  \
0            ANC                 SEA                    5  ...         408.0   
1            LAX                 PBI                   10  ...         741.0   
2            SFO                 CLT                   20  ...         811.0   
3            LAX                 MIA                   20  ...         756.0   
4            SEA                 ANC                   25  ...         259.0   

   ARRIVAL_DELAY  DIVERTED  CANCELLED  CANCELLATION_REASON  AIR_SYSTEM_DELAY  \
0          -22.0         0          0                  NaN               NaN   
1           -9.0         0          0                  NaN               NaN   
2            5.0         0          0                  NaN               NaN   
3           -9.0         0          0                  NaN               NaN   
4          -21.0         0          0                  NaN               NaN   

   SECURITY_DELAY  AIRLINE_DELAY  LATE_AIRCRAFT_DELAY  WEATHER_DELAY  
0             NaN            NaN                  NaN            NaN  
1             NaN            NaN                  NaN            NaN  
2             NaN            NaN                  NaN            NaN  
3             NaN            NaN                  NaN            NaN  
4             NaN            NaN                  NaN            NaN  

[5 rows x 31 columns]

What need to know what we're doing with the dataset. In this case, lets look at flight delays as a function of airline and destination. We could try and do everything with groupby's, but lets do things another way.

In [3]:
df = dfo.set_index(["DESTINATION_AIRPORT", "AIRLINE"])
df = df.sort_index()
df.head()

YEAR  MONTH  DAY  DAY_OF_WEEK  FLIGHT_NUMBER  \
DESTINATION_AIRPORT AIRLINE                                                 
ABE                 DL       2015      1    4            7           2036   
                    DL       2015      1    5            1           1411   
                    DL       2015      1    6            2           1411   
                    DL       2015      1    7            3           1411   
                    DL       2015      1    8            4           1411   

                            TAIL_NUMBER ORIGIN_AIRPORT  SCHEDULED_DEPARTURE  \
DESTINATION_AIRPORT AIRLINE                                                   
ABE                 DL           N979AT            ATL                 2101   
                    DL           N989AT            ATL                 2034   
                    DL           N979AT            ATL                 2034   
                    DL           N958AT            ATL                 2034   
                    DL           N994AT            ATL                 2034   

                             DEPARTURE_TIME  DEPARTURE_DELAY  ...  \
DESTINATION_AIRPORT AIRLINE                                   ...   
ABE                 DL               2101.0              0.0  ...   
                    DL               2032.0             -2.0  ...   
                    DL               2032.0             -2.0  ...   
                    DL               2031.0             -3.0  ...   
                    DL               2030.0             -4.0  ...   

                             ARRIVAL_TIME  ARRIVAL_DELAY  DIVERTED  CANCELLED  \
DESTINATION_AIRPORT AIRLINE                                                     
ABE                 DL             2252.0            1.0         0          0   
                    DL             2227.0            2.0         0          0   
                    DL             2221.0           -4.0         0          0   
                    DL             2226.0            1.0         0          0   
                    DL             2223.0           -2.0         0          0   

                             CANCELLATION_REASON  AIR_SYSTEM_DELAY  \
DESTINATION_AIRPORT AIRLINE                                          
ABE                 DL                       NaN               NaN   
                    DL                       NaN               NaN   
                    DL                       NaN               NaN   
                    DL                       NaN               NaN   
                    DL                       NaN               NaN   

                             SECURITY_DELAY  AIRLINE_DELAY  \
DESTINATION_AIRPORT AIRLINE                                  
ABE                 DL                  NaN            NaN   
                    DL                  NaN            NaN   
                    DL                  NaN            NaN   
                    DL                  NaN            NaN   
                    DL                  NaN            NaN   

                             LATE_AIRCRAFT_DELAY  WEATHER_DELAY  
DESTINATION_AIRPORT AIRLINE                                      
ABE                 DL                       NaN            NaN  
                    DL                       NaN            NaN  
                    DL                       NaN            NaN  
                    DL                       NaN            NaN  
                    DL                       NaN            NaN  

[5 rows x 29 columns]

Why sort? Feel free to remove that line and keep running this notebook yourself - at some point you'll get warnings from pandas that things might be slow. If you sort your index you trade the slow sorting method for far greater performance down the line. With a sorted index, looking up values can use intelligent algorithms. If its not sorted, it has to go row by row and check.

## Selecting data in a MultiIndex

Let's say we want to look at all the flights from AA flights going to Los Angeles

In [4]:
df_old = dfo[(dfo.AIRLINE == "AA") & (dfo.DESTINATION_AIRPORT == "LAX")]
df_old.head()

YEAR  MONTH  DAY  DAY_OF_WEEK AIRLINE  FLIGHT_NUMBER TAIL_NUMBER  \
443  2015      1    1            4      AA           1361      N3KYAA   
808  2015      1    1            4      AA           2401      N858AA   
867  2015      1    1            4      AA              9      N792AA   
871  2015      1    1            4      AA            201      N3LJAA   
880  2015      1    1            4      AA           1529      N5DHAA   

    ORIGIN_AIRPORT DESTINATION_AIRPORT  SCHEDULED_DEPARTURE  ...  \
443            BNA                 LAX                  610  ...   
808            DFW                 LAX                  655  ...   
867            JFK                 LAX                  700  ...   
871            BOS                 LAX                  700  ...   
880            MCO                 LAX                  700  ...   

     ARRIVAL_TIME  ARRIVAL_DELAY  DIVERTED  CANCELLED  CANCELLATION_REASON  \
443         847.0           -8.0         0          0                  NaN   
808           NaN            NaN         0          1                    B   
867        1026.0            6.0         0          0                  NaN   
871        1045.0           10.0         0          0                  NaN   
880         926.0           -9.0         0          0                  NaN   

     AIR_SYSTEM_DELAY  SECURITY_DELAY  AIRLINE_DELAY  LATE_AIRCRAFT_DELAY  \
443               NaN             NaN            NaN                  NaN   
808               NaN             NaN            NaN                  NaN   
867               NaN             NaN            NaN                  NaN   
871               NaN             NaN            NaN                  NaN   
880               NaN             NaN            NaN                  NaN   

     WEATHER_DELAY  
443            NaN  
808            NaN  
867            NaN  
871            NaN  
880            NaN  

[5 rows x 31 columns]

In [6]:
df_new = df.xs(("LAX", "AA"))
df_new.head()

YEAR  MONTH  DAY  DAY_OF_WEEK  FLIGHT_NUMBER  \
DESTINATION_AIRPORT AIRLINE                                                 
LAX                 AA       2015      1    1            4           1361   
                    AA       2015      1    1            4           2401   
                    AA       2015      1    1            4              9   
                    AA       2015      1    1            4            201   
                    AA       2015      1    1            4           1529   

                            TAIL_NUMBER ORIGIN_AIRPORT  SCHEDULED_DEPARTURE  \
DESTINATION_AIRPORT AIRLINE                                                   
LAX                 AA           N3KYAA            BNA                  610   
                    AA           N858AA            DFW                  655   
                    AA           N792AA            JFK                  700   
                    AA           N3LJAA            BOS                  700   
                    AA           N5DHAA            MCO                  700   

                             DEPARTURE_TIME  DEPARTURE_DELAY  ...  \
DESTINATION_AIRPORT AIRLINE                                   ...   
LAX                 AA                607.0             -3.0  ...   
                    AA                  NaN              NaN  ...   
                    AA                649.0            -11.0  ...   
                    AA                653.0             -7.0  ...   
                    AA                658.0             -2.0  ...   

                             ARRIVAL_TIME  ARRIVAL_DELAY  DIVERTED  CANCELLED  \
DESTINATION_AIRPORT AIRLINE                                                     
LAX                 AA              847.0           -8.0         0          0   
                    AA                NaN            NaN         0          1   
                    AA             1026.0            6.0         0          0   
                    AA             1045.0           10.0         0          0   
                    AA              926.0           -9.0         0          0   

                             CANCELLATION_REASON  AIR_SYSTEM_DELAY  \
DESTINATION_AIRPORT AIRLINE                                          
LAX                 AA                       NaN               NaN   
                    AA                         B               NaN   
                    AA                       NaN               NaN   
                    AA                       NaN               NaN   
                    AA                       NaN               NaN   

                             SECURITY_DELAY  AIRLINE_DELAY  \
DESTINATION_AIRPORT AIRLINE                                  
LAX                 AA                  NaN            NaN   
                    AA                  NaN            NaN   
                    AA                  NaN            NaN   
                    AA                  NaN            NaN   
                    AA                  NaN            NaN   

                             LATE_AIRCRAFT_DELAY  WEATHER_DELAY  
DESTINATION_AIRPORT AIRLINE                                      
LAX                 AA                       NaN            NaN  
                    AA                       NaN            NaN  
                    AA                       NaN            NaN  
                    AA                       NaN            NaN  
                    AA                       NaN            NaN  

[5 rows x 29 columns]

In [7]:
df.loc[("LAX", "AA")].head()

YEAR  MONTH  DAY  DAY_OF_WEEK  FLIGHT_NUMBER  \
DESTINATION_AIRPORT AIRLINE                                                 
LAX                 AA       2015      1    1            4           1361   
                    AA       2015      1    1            4           2401   
                    AA       2015      1    1            4              9   
                    AA       2015      1    1            4            201   
                    AA       2015      1    1            4           1529   

                            TAIL_NUMBER ORIGIN_AIRPORT  SCHEDULED_DEPARTURE  \
DESTINATION_AIRPORT AIRLINE                                                   
LAX                 AA           N3KYAA            BNA                  610   
                    AA           N858AA            DFW                  655   
                    AA           N792AA            JFK                  700   
                    AA           N3LJAA            BOS                  700   
                    AA           N5DHAA            MCO                  700   

                             DEPARTURE_TIME  DEPARTURE_DELAY  ...  \
DESTINATION_AIRPORT AIRLINE                                   ...   
LAX                 AA                607.0             -3.0  ...   
                    AA                  NaN              NaN  ...   
                    AA                649.0            -11.0  ...   
                    AA                653.0             -7.0  ...   
                    AA                658.0             -2.0  ...   

                             ARRIVAL_TIME  ARRIVAL_DELAY  DIVERTED  CANCELLED  \
DESTINATION_AIRPORT AIRLINE                                                     
LAX                 AA              847.0           -8.0         0          0   
                    AA                NaN            NaN         0          1   
                    AA             1026.0            6.0         0          0   
                    AA             1045.0           10.0         0          0   
                    AA              926.0           -9.0         0          0   

                             CANCELLATION_REASON  AIR_SYSTEM_DELAY  \
DESTINATION_AIRPORT AIRLINE                                          
LAX                 AA                       NaN               NaN   
                    AA                         B               NaN   
                    AA                       NaN               NaN   
                    AA                       NaN               NaN   
                    AA                       NaN               NaN   

                             SECURITY_DELAY  AIRLINE_DELAY  \
DESTINATION_AIRPORT AIRLINE                                  
LAX                 AA                  NaN            NaN   
                    AA                  NaN            NaN   
                    AA                  NaN            NaN   
                    AA                  NaN            NaN   
                    AA                  NaN            NaN   

                             LATE_AIRCRAFT_DELAY  WEATHER_DELAY  
DESTINATION_AIRPORT AIRLINE                                      
LAX                 AA                       NaN            NaN  
                    AA                       NaN            NaN  
                    AA                       NaN            NaN  
                    AA                       NaN            NaN  
                    AA                       NaN            NaN  

[5 rows x 29 columns]

In [9]:
df.FLIGHT_NUMBER["LAX", "AA"]

(LAX, AA)    1361
(LAX, AA)    2401
(LAX, AA)       9
(LAX, AA)     201
(LAX, AA)    1529
             ... 
(LAX, AA)     298
(LAX, AA)    2491
(LAX, AA)     246
(LAX, AA)      14
(LAX, AA)     286
Name: FLIGHT_NUMBER, Length: 2633, dtype: int64

In [10]:
df.FLIGHT_NUMBER[("LAX", "AA")]

(LAX, AA)    1361
(LAX, AA)    2401
(LAX, AA)       9
(LAX, AA)     201
(LAX, AA)    1529
             ... 
(LAX, AA)     298
(LAX, AA)    2491
(LAX, AA)     246
(LAX, AA)      14
(LAX, AA)     286
Name: FLIGHT_NUMBER, Length: 2633, dtype: int64

In [11]:
df_int = df.loc["AAA":]
df_int.head()

YEAR  MONTH  DAY  DAY_OF_WEEK  FLIGHT_NUMBER  \
DESTINATION_AIRPORT AIRLINE                                                 
ABE                 DL       2015      1    4            7           2036   
                    DL       2015      1    5            1           1411   
                    DL       2015      1    6            2           1411   
                    DL       2015      1    7            3           1411   
                    DL       2015      1    8            4           1411   

                            TAIL_NUMBER ORIGIN_AIRPORT  SCHEDULED_DEPARTURE  \
DESTINATION_AIRPORT AIRLINE                                                   
ABE                 DL           N979AT            ATL                 2101   
                    DL           N989AT            ATL                 2034   
                    DL           N979AT            ATL                 2034   
                    DL           N958AT            ATL                 2034   
                    DL           N994AT            ATL                 2034   

                             DEPARTURE_TIME  DEPARTURE_DELAY  ...  \
DESTINATION_AIRPORT AIRLINE                                   ...   
ABE                 DL               2101.0              0.0  ...   
                    DL               2032.0             -2.0  ...   
                    DL               2032.0             -2.0  ...   
                    DL               2031.0             -3.0  ...   
                    DL               2030.0             -4.0  ...   

                             ARRIVAL_TIME  ARRIVAL_DELAY  DIVERTED  CANCELLED  \
DESTINATION_AIRPORT AIRLINE                                                     
ABE                 DL             2252.0            1.0         0          0   
                    DL             2227.0            2.0         0          0   
                    DL             2221.0           -4.0         0          0   
                    DL             2226.0            1.0         0          0   
                    DL             2223.0           -2.0         0          0   

                             CANCELLATION_REASON  AIR_SYSTEM_DELAY  \
DESTINATION_AIRPORT AIRLINE                                          
ABE                 DL                       NaN               NaN   
                    DL                       NaN               NaN   
                    DL                       NaN               NaN   
                    DL                       NaN               NaN   
                    DL                       NaN               NaN   

                             SECURITY_DELAY  AIRLINE_DELAY  \
DESTINATION_AIRPORT AIRLINE                                  
ABE                 DL                  NaN            NaN   
                    DL                  NaN            NaN   
                    DL                  NaN            NaN   
                    DL                  NaN            NaN   
                    DL                  NaN            NaN   

                             LATE_AIRCRAFT_DELAY  WEATHER_DELAY  
DESTINATION_AIRPORT AIRLINE                                      
ABE                 DL                       NaN            NaN  
                    DL                       NaN            NaN  
                    DL                       NaN            NaN  
                    DL                       NaN            NaN  
                    DL                       NaN            NaN  

[5 rows x 29 columns]

In [12]:
df_int.loc[("ABE", "AA"):("ABZ", "MM")]

YEAR  MONTH  DAY  DAY_OF_WEEK  FLIGHT_NUMBER  \
DESTINATION_AIRPORT AIRLINE                                                 
ABE                 DL       2015      1    4            7           2036   
                    DL       2015      1    5            1           1411   
                    DL       2015      1    6            2           1411   
                    DL       2015      1    7            3           1411   
                    DL       2015      1    8            4           1411   
...                           ...    ...  ...          ...            ...   
ABY                 EV       2015      1   30            5           5334   
                    EV       2015      1   30            5           5335   
                    EV       2015      1   30            5           5351   
                    EV       2015      1   31            6           5334   
                    EV       2015      1   31            6           5351   

                            TAIL_NUMBER ORIGIN_AIRPORT  SCHEDULED_DEPARTURE  \
DESTINATION_AIRPORT AIRLINE                                                   
ABE                 DL           N979AT            ATL                 2101   
                    DL           N989AT            ATL                 2034   
                    DL           N979AT            ATL                 2034   
                    DL           N958AT            ATL                 2034   
                    DL           N994AT            ATL                 2034   
...                                 ...            ...                  ...   
ABY                 EV           N920EV            ATL                 1028   
                    EV           N868AS            ATL                 1605   
                    EV           N867AS            ATL                 2046   
                    EV           N900EV            ATL                 1253   
                    EV           N883AS            ATL                 2046   

                             DEPARTURE_TIME  DEPARTURE_DELAY  ...  \
DESTINATION_AIRPORT AIRLINE                                   ...   
ABE                 DL               2101.0              0.0  ...   
                    DL               2032.0             -2.0  ...   
                    DL               2032.0             -2.0  ...   
                    DL               2031.0             -3.0  ...   
                    DL               2030.0             -4.0  ...   
...                                     ...              ...  ...   
ABY                 EV               1022.0             -6.0  ...   
                    EV               1557.0             -8.0  ...   
                    EV               2047.0              1.0  ...   
                    EV               1250.0             -3.0  ...   
                    EV               2241.0            115.0  ...   

                             ARRIVAL_TIME  ARRIVAL_DELAY  DIVERTED  CANCELLED  \
DESTINATION_AIRPORT AIRLINE                                                     
ABE                 DL             2252.0            1.0         0          0   
                    DL             2227.0            2.0         0          0   
                    DL             2221.0           -4.0         0          0   
                    DL             2226.0            1.0         0          0   
                    DL             2223.0           -2.0         0          0   
...                                   ...            ...       ...        ...   
ABY                 EV             1114.0           -7.0         0          0   
                    EV             1637.0          -18.0         0          0   
                    EV             2130.0           -7.0         0          0   
                    EV             1335.0           -8.0         0          0   
                    EV             2327.0          111.0         0          0   

                             CANCELLATION_REASO

In [15]:
df_int.loc[(slice(None), "AA"), :]

YEAR  MONTH  DAY  DAY_OF_WEEK  FLIGHT_NUMBER  \
DESTINATION_AIRPORT AIRLINE                                                 
ABQ                 AA       2015      1    1            4           1662   
                    AA       2015      1    1            4           1024   
                    AA       2015      1    1            4           1203   
                    AA       2015      1    1            4           2388   
                    AA       2015      1    2            5           1662   
...                           ...    ...  ...          ...            ...   
TUS                 AA       2015      1   31            6           1261   
                    AA       2015      1   31            6           2304   
                    AA       2015      1   31            6           1643   
                    AA       2015      1   31            6           1001   
                    AA       2015      1   31            6            301   

                            TAIL_NUMBER ORIGIN_AIRPORT  SCHEDULED_DEPARTURE  \
DESTINATION_AIRPORT AIRLINE                                                   
ABQ                 AA           N4WJAA            DFW                 1045   
                    AA           N505AA            DFW                 1400   
                    AA           N4XPAA            DFW                 1840   
                    AA           N486AA            DFW                 2115   
                    AA           N582AA            DFW                 1045   
...                                 ...            ...                  ...   
TUS                 AA           N475AA            DFW                 1050   
                    AA           N556AA            DFW                 1240   
                    AA           N4XCAA            DFW                 1450   
                    AA           N4UBAA            DFW                 1830   
                    AA           N4YJAA            DFW                 2005   

                             DEPARTURE_TIME  DEPARTURE_DELAY  ...  \
DESTINATION_AIRPORT AIRLINE                                   ...   
ABQ                 AA                  NaN              NaN  ...   
                    AA               1408.0              8.0  ...   
                    AA               1853.0             13.0  ...   
                    AA               2126.0             11.0  ...   
                    AA               1044.0             -1.0  ...   
...                                     ...              ...  ...   
TUS                 AA               1135.0             45.0  ...   
                    AA               1235.0             -5.0  ...   
                    AA               1445.0             -5.0  ...   
                    AA               1906.0             36.0  ...   
                    AA               2001.0             -4.0  ...   

                             ARRIVAL_TIME  ARRIVAL_DELAY  DIVERTED  CANCELLED  \
DESTINATION_AIRPORT AIRLINE                                                     
ABQ                 AA                NaN            NaN         0          1   
                    AA             1500.0           10.0         0          0   
                    AA             1953.0           23.0         0          0   
                    AA             2210.0           10.0         0          0   
                    AA             1126.0           -9.0         0          0   
...                                   ...            ...       ...        ...   
TUS                 AA             1256.0           39.0         0          0   
                    AA             1348.0          -18.0         0          0   
                    AA             1558.0          -14.0         0          0   
                    AA             2023.0           29.0         0          0   
                    AA             2129.0           -1.0         0          0   

                             CANCELLATION_REASO

In [16]:
df_int.loc[(slice("A", "B"), slice("D", "F")), :]

YEAR  MONTH  DAY  DAY_OF_WEEK  FLIGHT_NUMBER  \
DESTINATION_AIRPORT AIRLINE                                                 
ABE                 DL       2015      1    4            7           2036   
                    DL       2015      1    5            1           1411   
                    DL       2015      1    6            2           1411   
                    DL       2015      1    7            3           1411   
                    DL       2015      1    8            4           1411   
...                           ...    ...  ...          ...            ...   
AZO                 EV       2015      1   29            4           4997   
                    EV       2015      1   30            5           5386   
                    EV       2015      1   30            5           4997   
                    EV       2015      1   31            6           5438   
                    EV       2015      1   31            6           4945   

                            TAIL_NUMBER ORIGIN_AIRPORT  SCHEDULED_DEPARTURE  \
DESTINATION_AIRPORT AIRLINE                                                   
ABE                 DL           N979AT            ATL                 2101   
                    DL           N989AT            ATL                 2034   
                    DL           N979AT            ATL                 2034   
                    DL           N958AT            ATL                 2034   
                    DL           N994AT            ATL                 2034   
...                                 ...            ...                  ...   
AZO                 EV           N528CA            DTW                 2140   
                    EV           N924EV            DTW                 1215   
                    EV           N854AS            DTW                 2140   
                    EV           N611QX            DTW                 1545   
                    EV           N868AS            ATL                 1945   

                             DEPARTURE_TIME  DEPARTURE_DELAY  ...  \
DESTINATION_AIRPORT AIRLINE                                   ...   
ABE                 DL               2101.0              0.0  ...   
                    DL               2032.0             -2.0  ...   
                    DL               2032.0             -2.0  ...   
                    DL               2031.0             -3.0  ...   
                    DL               2030.0             -4.0  ...   
...                                     ...              ...  ...   
AZO                 EV               2140.0              0.0  ...   
                    EV               1211.0             -4.0  ...   
                    EV               2200.0             20.0  ...   
                    EV               1541.0             -4.0  ...   
                    EV               1932.0            -13.0  ...   

                             ARRIVAL_TIME  ARRIVAL_DELAY  DIVERTED  CANCELLED  \
DESTINATION_AIRPORT AIRLINE                                                     
ABE                 DL             2252.0            1.0         0          0   
                    DL             2227.0            2.0         0          0   
                    DL             2221.0           -4.0         0          0   
                    DL             2226.0            1.0         0          0   
                    DL             2223.0           -2.0         0          0   
...                                   ...            ...       ...        ...   
AZO                 EV                NaN            NaN         0          1   
                    EV             1254.0          -18.0         0          0   
                    EV             2242.0            9.0         0          0   
                    EV             1636.0           -2.0         0          0   
                    EV             2112.0          -30.0         0          0   

                             CANCELLATION_REASO

In [17]:
idx = pd.IndexSlice
df_int.loc[idx[:, "AA"], :]

YEAR  MONTH  DAY  DAY_OF_WEEK  FLIGHT_NUMBER  \
DESTINATION_AIRPORT AIRLINE                                                 
ABQ                 AA       2015      1    1            4           1662   
                    AA       2015      1    1            4           1024   
                    AA       2015      1    1            4           1203   
                    AA       2015      1    1            4           2388   
                    AA       2015      1    2            5           1662   
...                           ...    ...  ...          ...            ...   
TUS                 AA       2015      1   31            6           1261   
                    AA       2015      1   31            6           2304   
                    AA       2015      1   31            6           1643   
                    AA       2015      1   31            6           1001   
                    AA       2015      1   31            6            301   

                            TAIL_NUMBER ORIGIN_AIRPORT  SCHEDULED_DEPARTURE  \
DESTINATION_AIRPORT AIRLINE                                                   
ABQ                 AA           N4WJAA            DFW                 1045   
                    AA           N505AA            DFW                 1400   
                    AA           N4XPAA            DFW                 1840   
                    AA           N486AA            DFW                 2115   
                    AA           N582AA            DFW                 1045   
...                                 ...            ...                  ...   
TUS                 AA           N475AA            DFW                 1050   
                    AA           N556AA            DFW                 1240   
                    AA           N4XCAA            DFW                 1450   
                    AA           N4UBAA            DFW                 1830   
                    AA           N4YJAA            DFW                 2005   

                             DEPARTURE_TIME  DEPARTURE_DELAY  ...  \
DESTINATION_AIRPORT AIRLINE                                   ...   
ABQ                 AA                  NaN              NaN  ...   
                    AA               1408.0              8.0  ...   
                    AA               1853.0             13.0  ...   
                    AA               2126.0             11.0  ...   
                    AA               1044.0             -1.0  ...   
...                                     ...              ...  ...   
TUS                 AA               1135.0             45.0  ...   
                    AA               1235.0             -5.0  ...   
                    AA               1445.0             -5.0  ...   
                    AA               1906.0             36.0  ...   
                    AA               2001.0             -4.0  ...   

                             ARRIVAL_TIME  ARRIVAL_DELAY  DIVERTED  CANCELLED  \
DESTINATION_AIRPORT AIRLINE                                                     
ABQ                 AA                NaN            NaN         0          1   
                    AA             1500.0           10.0         0          0   
                    AA             1953.0           23.0         0          0   
                    AA             2210.0           10.0         0          0   
                    AA             1126.0           -9.0         0          0   
...                                   ...            ...       ...        ...   
TUS                 AA             1256.0           39.0         0          0   
                    AA             1348.0          -18.0         0          0   
                    AA             1558.0          -14.0         0          0   
                    AA             2023.0           29.0         0          0   
                    AA             2129.0           -1.0         0          0   

                             CANCELLATION_REASO

In [19]:
df_int.loc[idx["A":"B", "D":"F"], :]

YEAR  MONTH  DAY  DAY_OF_WEEK  FLIGHT_NUMBER  \
DESTINATION_AIRPORT AIRLINE                                                 
ABE                 DL       2015      1    4            7           2036   
                    DL       2015      1    5            1           1411   
                    DL       2015      1    6            2           1411   
                    DL       2015      1    7            3           1411   
                    DL       2015      1    8            4           1411   
...                           ...    ...  ...          ...            ...   
AZO                 EV       2015      1   29            4           4997   
                    EV       2015      1   30            5           5386   
                    EV       2015      1   30            5           4997   
                    EV       2015      1   31            6           5438   
                    EV       2015      1   31            6           4945   

                            TAIL_NUMBER ORIGIN_AIRPORT  SCHEDULED_DEPARTURE  \
DESTINATION_AIRPORT AIRLINE                                                   
ABE                 DL           N979AT            ATL                 2101   
                    DL           N989AT            ATL                 2034   
                    DL           N979AT            ATL                 2034   
                    DL           N958AT            ATL                 2034   
                    DL           N994AT            ATL                 2034   
...                                 ...            ...                  ...   
AZO                 EV           N528CA            DTW                 2140   
                    EV           N924EV            DTW                 1215   
                    EV           N854AS            DTW                 2140   
                    EV           N611QX            DTW                 1545   
                    EV           N868AS            ATL                 1945   

                             DEPARTURE_TIME  DEPARTURE_DELAY  ...  \
DESTINATION_AIRPORT AIRLINE                                   ...   
ABE                 DL               2101.0              0.0  ...   
                    DL               2032.0             -2.0  ...   
                    DL               2032.0             -2.0  ...   
                    DL               2031.0             -3.0  ...   
                    DL               2030.0             -4.0  ...   
...                                     ...              ...  ...   
AZO                 EV               2140.0              0.0  ...   
                    EV               1211.0             -4.0  ...   
                    EV               2200.0             20.0  ...   
                    EV               1541.0             -4.0  ...   
                    EV               1932.0            -13.0  ...   

                             ARRIVAL_TIME  ARRIVAL_DELAY  DIVERTED  CANCELLED  \
DESTINATION_AIRPORT AIRLINE                                                     
ABE                 DL             2252.0            1.0         0          0   
                    DL             2227.0            2.0         0          0   
                    DL             2221.0           -4.0         0          0   
                    DL             2226.0            1.0         0          0   
                    DL             2223.0           -2.0         0          0   
...                                   ...            ...       ...        ...   
AZO                 EV                NaN            NaN         0          1   
                    EV             1254.0          -18.0         0          0   
                    EV             2242.0            9.0         0          0   
                    EV             1636.0           -2.0         0          0   
                    EV             2112.0          -30.0         0          0   

                             CANCELLATION_REASO

Out of time on this one! Next lecture we'll go into stacking, unstacking, MultiIndexes on the column and other fun things you can do! Now I hope you've been going through all the videos in the order laid out, because you've seen MultiIndexing before.

### Recap
* MultiIndex creation
* loc and xs with MultiIndex
* slice & pd.IndexSlice